In [27]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 13 15:13:55 2024

@author: Sezer
"""
import numpy as np
import networkx as nx
import pandas as pd
from rdkit import Chem

# Atomic properties table
atomicProperties = {
    "C": {"AtomicMass": 12.011, "AtomicRadius": 70, "VanDerWaals": 170,"Electronegativity": 2.55, "AtomicNumber": 6, "Density": 2.260, "Polarizability": 1.76,"BoilingPoint": 4098,"Ionization":11.260},
    "N": {"AtomicMass": 14.007, "AtomicRadius": 65, "VanDerWaals": 155, "Electronegativity": 3.12, "AtomicNumber": 7, "Density": 1.251, "Polarizability": 1.10,"BoilingPoint": 77.36,"Ionization":14.534},
    "O": {"AtomicMass": 15.999, "AtomicRadius": 60,"VanDerWaals": 152, "Electronegativity": 3.62, "AtomicNumber": 8, "Density": 1.429, "Polarizability": 0.802,"BoilingPoint":90.2,"Ionization":13.618},
    "F": {"AtomicMass": 18.998, "AtomicRadius": 50, "VanDerWaals": 135, "Electronegativity": 4.23, "AtomicNumber": 9, "Density": 1.696, "Polarizability": 0.557,"BoilingPoint":85.03,"Ionization":17.423},
    "S": {"AtomicMass": 32.066, "AtomicRadius": 100,"VanDerWaals": 180, "Electronegativity": 2.49, "AtomicNumber": 16, "Density": 1.960, "Polarizability": 2.9,"BoilingPoint":717.75,"Ionization":10.36},
    "Cl": {"AtomicMass": 35.453, "AtomicRadius": 100, "VanDerWaals": 175,"Electronegativity": 2.82, "AtomicNumber": 17, "Density": 3.214, "Polarizability": 2.82,"BoilingPoint":239.11,"Ionization":12.968},
    "Pt": {"AtomicMass": 195.079, "AtomicRadius": 135, "VanDerWaals": 209,"Electronegativity": 2.28, "AtomicNumber": 78, "Density": 21.090, "Polarizability": None,"BoilingPoint":4097.15,"Ionization":9},
    "Br": {"AtomicMass": 79.904, "AtomicRadius": 115, "VanDerWaals": 183,"Electronegativity": 2.56,"AtomicNumber": 35,"Density": 3.12,"Polarizability": None,"BoilingPoint":331.95,"Ionization":11.814},
    "P": {"AtomicMass": 30.97, "AtomicRadius": 100,"VanDerWaals": 180, "Electronegativity": 2.22,"AtomicNumber": 15,"Density": 1.823,"Polarizability": 3.63,"BoilingPoint":553.65,"Ionization":10.487},
}

# Function to calculate vertex weight based on atomic properties
def calculate_vertex_weight(atom):
    atomic_AtomicMass = atomicProperties[atom]["AtomicMass"]
    return 1 - (atomicProperties["C"]["AtomicMass"] / atomic_AtomicMass)

# Function to calculate edge weight based on atomic properties
def calculate_edge_weight(bond):
    atom1, atom2, bond_type = bond.GetBeginAtom().GetSymbol(), bond.GetEndAtom().GetSymbol(), bond.GetBondTypeAsDouble()
    atomic_AtomicMass1 = atomicProperties[atom1]["AtomicMass"]
    atomic_AtomicMass2 = atomicProperties[atom2]["AtomicMass"]
    bond_order = int(bond_type)
    return (atomicProperties["C"]["AtomicMass"] * atomicProperties["C"]["AtomicMass"]) / (
            bond_order * atomic_AtomicMass1 * atomic_AtomicMass2)

# Function to convert graph to adjacency matrix
def graph_to_adjacency_matrix(graph):
    num_nodes = len(graph.nodes)
    adjacency_matrix = np.zeros((num_nodes, num_nodes))
    for u, v, data in graph.edges(data=True):
        adjacency_matrix[u][v] = data['weight']
        adjacency_matrix[v][u] = data['weight']  # Assuming undirected graph
    
    # Set diagonal elements to vertex weights
    for node in graph.nodes(data=True):
        node_index = node[0]
        vertex_weight = node[1]['weight']
        adjacency_matrix[node_index][node_index] = vertex_weight
    
    return adjacency_matrix

# List of SMILES strings representing chemical graphs
smiles_list =["CN(C)CCCN1c2ccccc2Sc2ccc(Cl)cc21",
              "CN1CCN(CCCN2c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1",
              "CSc1ccc2c(c1)N(CCC1CCCCN1C)c1ccccc1S2", 
              "CN1CCN(CC/C=C2/c3ccccc3Sc3ccc(S(=O)(=O)N(C)C)cc32)CC1",
              "O=C(CCCN1CCC(O)(c2ccc(Cl)cc2)CC1)c1ccc(F)cc1",
            "CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1",
              "O=C1Cc2cc(CCN3CCN(c4nsc5ccccc45)CC3)c(Cl)cc2N1",
              "CN1CCN(C2=Nc3ccccc3Oc3ccc(Cl)cc32)CC1", 
              "OCCOCCN1CCN(C2=Nc3ccccc3Sc3ccccc32)CC1",
              "O=C1CCc2ccc(OCCCCN3CCN(c4cccc(Cl)c4Cl)CC3)cc2N1",
              "Cc1nc2n(c(=O)c1CCN1CCC(c3noc4cc(F)ccc34)CC1)CCCC2", 
              "Cc1cc2c(s1)Nc1ccccc1N=C2N1CCN(C)CC1",


"COc1ccc(-n2nc(C(N)=O)c3c2C(=O)N(c2ccc(N4CCCCC4=O)cc2)CC3)cc1",
"COC(=O)Nc1c(N)nc(-c2nn(Cc3ccccc3F)c3ncc(F)cc23)nc1N",
"CCCCCCOC(=O)N=C(N)c1ccc(NCc2nc3cc(C(=O)N(CCC(=O)OCC)c4ccccn4)ccc3n2C)cc1",
"COc1cc2c(cc1OC)CC(=O)N(CCCN(C)C[C@H]1Cc3cc(OC)c(OC)cc31)CC2",
"CCOc1ccc(Cc2cc([C@@H]3O[C@H](CO)[C@@H](O)[C@H](O)[C@H]3O)ccc2Cl)cc1",
"OC[C@H]1O[C@@H](c2ccc(Cl)c(Cc3ccc(O[C@H]4CCOC4)cc3)c2)[C@H](O)[C@@H](O)[C@@H]1O",
"COCCc1ccc(OCC(O)CNC(C)C)cc1", 
"CCOC(=O)[C@H](C)C[C@@H](Cc1ccc(-c2ccccc2)cc1)NC(=O)CCC(=O)O", 
"CCCCC(=O)N(Cc1ccc(-c2ccccc2-c2nnn[nH]2)cc1)[C@H](C(=O)O)C(C)C",


"C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O[C@H]3[C@H](O)[C@@H](O)[C@H](O)O[C@@H]3CO)O[C@@H]2CO)[C@H](O)[C@@H](O)[C@@H]1N[C@H]1C=C(CO)[C@@H](O)[C@H](O)[C@H]1O",
"Cc1ccc(S(=O)(=O)NC(O)=NN2CCCCCC2)cc1",
"OCCN1C[C@H](O)[C@@H](O)[C@H](O)[C@H]1CO",
"CCOc1cc(CC(=O)N[C@@H](CC(C)C)c2ccccc2N2CCCCC2)ccc1C(=O)O",
"CN(C)C(=N)NC(=N)N",
"CC#CCn1c(N2CCC[C@@H](N)C2)nc2c1c(=O)n(Cc1nc(C)c3ccccc3n1)c(=O)n2C",
"CCc1ccc(CCOc2ccc(CC3SC(=O)NC3=O)cc2)nc1",
"CC(C)C[C@H]1C(=O)N2CCC[C@H]2[C@]2(O)O[C@](NC(=O)[C@@H]3C=C4c5cccc6[nH]c(Br)c(c56)C[C@H]4N(C)C3)(C(C)C)C(=O)N12",
"Cn1c(=O)cc(N2CCC[C@@H](N)C2)n(Cc2ccccc2C#N)c1=O",

"CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12",
"CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O",
"OC(c1cc(C(F)(F)F)nc2c(C(F)(F)F)cccc12)C1CCCCN1",
"Clc1ccc2c(N3CCN(CCCN4CCN(c5ccnc6cc(Cl)ccc56)CC4)CC3)ccnc2c1",
"COc1cc(NC(C)CCCN)c2ncccc2c1",
"CCCCN(CCCC)CC(O)c1cc(Cl)cc2c1-c1ccc(Cl)cc1/C2=C/c1ccc(Cl)cc1",
"O=C1C(O)=C([C@H]2CC[C@H](c3ccc(Cl)cc3)CC2)C(=O)c2ccccc21",
"CCc1nc(=N)[nH]c(N)c1-c1ccc(Cl)cc1",
"C[C@H]1c2cccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(C(N)=O)=C(O)[C@@H](N(C)C)[C@@H]3[C@@H](O)[C@@H]21",


"Cn1cnc([N+](=O)[O-])c1Sc1ncnc2nc[nH]c12",
"CCN(CCO)CCCC(C)Nc1ccnc2cc(Cl)ccc12",
"O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O",
"O=C(Nc1nc2cccc(-c3ccc(CN4CCS(=O)(=O)CC4)cc3)n2n1)C1CC1",
"Cc1oncc1C(=O)Nc1ccc(C(F)(F)F)cc1",
"C[C@]12C=CC(=O)C=C1CC[C@@H]1[C@@H]2[C@@H](O)C[C@@]2(C)[C@H]1CC[C@]2(O)C(=O)CO",
"CN(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)N[C@@H](CCC(=O)O)C(=O)O)cc1",
"CCS(=O)(=O)N1CC(CC#N)(n2cc(-c3ncnc4[nH]ccc34)cn2)C1",
"CC1CCN(C(=O)CC#N)CC1N(C)c1ncnc2[nH]ccc12",
"CC[C@@H]1CN(C(=O)NCC(F)(F)F)C[C@@H]1c1cnc2cnc3[nH]ccc3n12",

"CCC(=O)O[C@]1(C(=O)SCF)[C@H](C)C[C@H]2[C@@H]3C[C@H](F)C4=CC(=O)C=C[C@]4(C)[C@@]3(F)[C@@H](O)C[C@@]21C",
"C[C@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C@@]3(F)C(=O)C[C@]2(C)[C@@]1(O)C(=O)CCl",
"CC1(C)O[C@@H]2C[C@H]3[C@@H]4CCC5=CC(=O)C=C[C@]5(C)[C@H]4[C@@H](O)C[C@]3(C)[C@]2(C(=O)CO)O1",
"CCC(=O)O[C@]1(C(=O)CCl)[C@@H](C)C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C@@]3(F)[C@@H](O)C[C@@]21C",
"Cn1cnc([N+](=O)[O-])c1Sc1ncnc2nc[nH]c12",
"Oc1ccc(OCc2ccccc2)cc1",
"CCCCC(=O)O[C@]1(C(=O)CO)[C@@H](C)CC2C3CCC4=CC(=O)C=C[C@]4(C)[C@@]3(F)[C@@H](O)C[C@@]21C",
"O=c1ccc2cc3ccoc3cc2o1",
"CCCCC(=O)O[C@]1(C(=O)CO)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3[C@@H](O)C[C@@]21C",
"C[C@@H]1C[C@H]2[C@@H]3C[C@H](F)C4=CC(=O)C=C[C@]4(C)[C@@]3(F)[C@@H](O)C[C@]2(C)[C@@]1(O)C(=O)SCF",


"Nc1ccn(C[C@@H](CO)OCP(=O)(O)O)c(=O)n1",
"O=C(O)P(=O)(O)O",
"CC(C)Nc1nc2cc(Cl)c(Cl)cc2n1[C@H]1O[C@@H](CO)[C@H](O)[C@@H]1O",
"CC(C)[C@H](N)C(=O)OCC(CO)OCn1cnc2c(=O)nc(N)[nH]c21",


"COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)C=CCN1CCCCC1",
"COc1ccc(CN2C3CC2CN(c2ccc(-c4cc(OCC(C)(C)O)cn5ncc(C#N)c45)cn2)C3)cn1",
"CN1CCC(COc2cnc(-c3cccc(Cn4nc(-c5cccc(C#N)c5)ccc4=O)c3)nc2)CC1",
"C=CC(=O)N1CCN(c2nc(=O)n(-c3c(C)ccnc3C(C)C)c3nc(-c4c(O)cccc4F)c(F)cc23)[C@@H](C)C1",
"COc1cc([C@@H]2c3cc4c(cc3[C@@H](O[C@@H]3O[C@@H]5CO[C@@H](C)O[C@H]5[C@H](O)[C@H]3O)[C@H]3COC(=O)[C@H]23)OCO4)cc(OC)c1O",
"CCc1cc2c(cc1N1CCC(N3CCOCC3)CC1)C(C)(C)c1[nH]c3cc(C#N)ccc3c1C2=O",
"CC(=O)O[C@H]1C(=O)[C@@]2(C)[C@H]([C@H](OC(=O)c3ccccc3)[C@]3(O)C[C@H](OC(=O)[C@H](O)[C@@H](NC(=O)c4ccccc4)c4ccccc4)C(C)=C1C3(C)C)[C@]1(OC(C)=O)CO[C@@H]1C[C@@H]2O",
"CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2F)c(-c2ccnc(N)n2)s1",
"CN1CCN(c2ccc(C(=O)Nc3n[nH]c4ccc(Cc5cc(F)cc(F)c5)cc34)c(NC3CCOCC3)c2)CC1",
"C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)ccc(F)c1Cl",
"Cc1cc(Nc2ncc(Cl)c(Nc3ccccc3S(=O)(=O)C(C)C)n2)c(OC(C)C)cc1C1CCNCC1",
"C[C@H]1Oc2cc(cnc2N)-c2c(nn(C)c2C#N)CN(C)C(=O)c2ccc(F)cc21",
"CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1O[C@H]1CCOC1",
"CO[C@]1(C(=O)N[C@@H](C)c2ccc(-n3cc(F)cn3)nc2)CC[C@@H](c2nc(C)cc(Nc3cc(C)[nH]n3)n2)CC1",
"COc1cc(N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc(Cl)c(Nc2ccccc2P(C)(C)=O)n1",
"C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1",
"C=C(F)C(=O)N1CCN(c2nc(OC[C@@H]3CCCN3C)nc3c2CCN(c2cccc4cccc(Cl)c24)C3)C[C@@H]1CC#N",
"COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1",
"CCC1=C[C@@H]2CN(C1)Cc1c([nH]c3ccccc13)[C@@](C(=O)OC)(c1cc3c(cc1OC)N(C)[C@H]1[C@@](O)(C(=O)OC)[C@H](OC(C)=O)[C@]4(CC)C=CCN5CC[C@]31[C@@H]54)C2",
"Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)C2(F)F)c(=O)n1",
"CC(=O)O[C@@]12CO[C@@H]1C[C@H](O)[C@@]1(C)C(=O)[C@H](O)C3=C(C)[C@@H](OC(=O)[C@H](O)[C@@H](NC(=O)OC(C)(C)C)c4ccccc4)C[C@@](O)([C@@H](OC(=O)c4ccccc4)[C@H]21)C3(C)C",
"Nc1nc(=O)c2c(CCc3ccc(C(=O)N[C@@H](CCC(=O)O)C(=O)O)cc3)c[nH]c2[nH]1",



"COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1",
"C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1",
"C=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1OCCCN1CCOCC1",
"CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1O[C@H]1CCOC1",
"COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1",
"Cc1ccc(C(=O)N(CCCN)[C@@H](c2nc3cc(Cl)ccc3c(=O)n2Cc2ccccc2)C(C)C)cc1",


"N=c1c2c([nH]c3ccccc13)CCCC2",
"COc1cc2c(cc1OC)C(=O)C(CC1CCN(Cc3ccccc3)CC1)C2",
"CCN(C)C(=O)Oc1cccc([C@H](C)N(C)C)c1",
"COc1ccc2c3c1O[C@H]1C[C@@H](O)C=C[C@@]31CCN(C)C2",
"C/C=C1\[C@H]2C=C(C)C[C@]1(N)c1ccc(=O)[nH]c1C2",


"NCC[C@H](O)C(=O)N[C@@H]1C[C@H](N)[C@@H](O[C@H]2O[C@H](CN)[C@@H](O)[C@H](O)[C@H]2O)[C@H](O)[C@H]1O[C@H]1O[C@H](CO)[C@@H](O)[C@H](N)[C@H]1O",
"COc1nc2ccc(Br)cc2cc1[C@@H](c1ccccc1)[C@@](O)(CCN(C)C)c1cccc2ccccc12",
"CC(C)/N=c1\cc2n(-c3ccc(Cl)cc3)c3ccccc3nc-2cc1Nc1ccc(Cl)cc1",
"C[C@]1(COc2ccc(N3CCC(Oc4ccc(OC(F)(F)F)cc4)CC3)cc2)Cn2cc([N+](=O)[O-])nc2O1",
"CCC(CO)NCCNC(CC)CO",
"CCc1cc(C(=N)S)ccn1",
"C[C@@H](O)[C@H]1C(=O)N2C(C(=O)O)=C(SCCNC=N)C[C@H]12",
"C[C@H]1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23",
"CC(=O)NC[C@H]1CN(c2ccc(N3CCOCC3)c(F)c2)C(=O)O1",
"COc1c(N2C[C@@H]3CCCN[C@@H]3C2)c(F)cc2c(=O)c(C(=O)O)cn(C3CC3)c12",
"Nc1ccc(C(=O)O)c(O)c1",
"N=C(O)c1cnccn1",
"CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(c(/C=N/N5CCN(C)CC5)c(O)c4c3C2=O)NC(=O)/C(C)=C\C=C\[C@H](C)[C@H](O)[C@@H](C)[C@@H](O)[C@@H](C)[C@H](OC(C)=O)[C@@H]1C",
"O=C1NOCC1/N=C/c1ccc(/C=N/C2CONC2=O)cc1",


"CO/N=C(/C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)=O)CS[C@H]12)c1ccco1",
"CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccc(O)cc3)C(=O)N2[C@H]1C(=O)O",
"CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23",
"C[C@]12CCC(=O)C=C1CC[C@@H]1[C@@H]2[C@@H](O)C[C@@]2(C)[C@H]1CC[C@]2(O)C(=O)CO",
"COc1c(N2C[C@@H]3CCCN[C@@H]3C2)c(F)cc2c(=O)c(C(=O)O)cn(C3CC3)c12",
"CC1(C)O[C@@H]2C[C@H]3[C@@H]4C[C@H](F)C5=CC(=O)CC[C@]5(C)[C@H]4[C@@H](O)C[C@]3(C)[C@]2(C(=O)CO)O1",
"Nc1ccc(S(=O)(=O)c2ccc(N)cc2)cc1",


"CC(C)NCC(O)COc1cccc2ccccc12",
"COCCc1ccc(OCC(O)CNC(C)C)cc1",
"CC(C)NCC(O)COc1ccc(CC(N)=O)cc1",
"COc1ccccc1OCCNCC(O)COc1cccc2[nH]c3ccccc3c12",

"C1=NC(=NC(=O)N1[C@H]2[C@@H]([C@@H]([C@H](O2)CO)O)O)N",
"CS(=O)(=O)OCCCCOS(=O)(=O)C",
"C1=NC2=C(N1)C(=S)N=CN2",
"C1=NC2=C(N1)C(=S)N=C(N2)N",
"COC1=NC(=NC2=C1N=CN2[C@H]3[C@H]([C@@H]([C@H](O3)CO)O)O)N",
"C1=CN(C(=O)N=C1N)[C@H]2[C@H]([C@@H]([C@H](O2)CO)O)O",
"C1=NC2=C(N=C(N=C2N1[C@H]3[C@H]([C@@H]([C@H](O3)CO)O)F)Cl)N",
"CN1CCN(CC1)CCCOC2=C(C=C3C(=C2)N=CC(=C3NC4=CC(=C(C=C4Cl)Cl)OC)C#N)OC",
"CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(=N3)C)N4CCN(CC4)CCO",
"C1=CC(=CC=C1CC(C(=O)O)N)N(CCCl)CCCl",
"C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@@]4([C@]3([C@H](C[C@@]2([C@]1(C(=O)CO)O)C)O)F)C",
"C[C@H]1[C@H]([C@H](C[C@@H](O1)O[C@H]2C[C@@](CC3=C2C(=C4C(=C3O)C(=O)C5=C(C4=O)C(=CC=C5)OC)O)(C(=O)CO)O)N)O",
"CN1[C@@H]([C@@H]([C@]2(C1=O)CCC=N2)C3=CC(=C(C=C3Br)Br)OC)O",
"C1=CC(=CC=C1C(=O)NC(CCC(=O)O)C(=O)O)NCC2=CN=C3C(=N2)C(=NC(=N3)N)N",
"COC1C(N2C(=C(C(=C2Br)Br)Br)C(=O)N1)C3=C(NC4=C3C=C(C=C4)Br)Br",
"C(CCl)NC(=O)N(CCCl)N=O",
"CNC1=CC(=N)c2ccncc2C1=O",
"O=C(N2C(=O)C(O)(c1cc(O)c(Br)c(Br)c1)CC2)CCCCCCCCCCCCC",
"CNCCC1=C(C(=C(C=C1Br)Br)OC)Br",
"CC(=O)C[C@]1(c2c(cc(cc2Br)Br)NC1=O)O",
"COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)(C(C)=O)C[C@@H]3O[C@H]1C[C@H](N)[C@H](O)[C@H](C)O1",
"COc1cc2c(cc1OC)C1C(=O)c3ccc4c(c3OC1CO2)C=CC(C)(C)O4",
"CC(=O)NCCC1=CNC2=C1C=C(C=C2)OC",
"CN(C)c1ccc(O)c2c1C[C@H]1C[C@H]3[C@H](N(C)C)C(O)=C(C(N)=O)C(=O)[C@@]3(O)C(O)=C1C2=O",
"CN1C=C2C(=CC1=O)C(=O)C(=C(C2=O)N)SC",
"COC1=CN2C(=CC1=O)C(=O)C3=C2C(=NC=C3)CC4=CC=CC=C4",
"C1CCN(CC1)CCOC2=CC=C(C=C2)C(=O)C3=C(SC4=C3C=CC(=C4)O)C5=CC=C(C=C5)O",
"CCN1CCN(CC1)CC2=CN=C(C=C2)NC3=NC=C(C(=N3)C4=CC5=C(C(=C4)F)N=C(N5C(C)C)C)F",
"CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@]([C@H]3[C@@H]([C@@](C2(C)C)(C[C@@H]1OC(=O)[C@@H]([C@H](C5=CC=CC=C5)NC(=O)C6=CC=CC=C6)O)O)OC(=O)C7=CC=CC=C7)(CO4)OC(=O)C)O)C)OC(=O)C",
"CC(C)(C#N)C1=CC(=CC(=C1)CN2C=NC=N2)C(C)(C)C#N",
"CCCCCOC(=O)NC1=NC(=O)N(C=C1F)[C@H]2[C@@H]([C@@H]([C@H](O2)C)O)O",
"O=P1(N(CCCl)CCCl)NCCCO1",
"C[C@@H]1CC[C@H]2C[C@@H](/C(=C/C=C/C=C/[C@H](C[C@H](C(=O)[C@@H]([C@@H](/C(=C/[C@H](C(=O)C[C@H](OC(=O)[C@@H]3CCCCN3C(=O)C(=O)[C@@]1(O2)O)[C@H](C)C[C@@H]4CC[C@H]([C@@H](C4)OC)OCCO)C)/C)O)OC)C)C)/C)OC",
"C[C@]12CC[C@H]3[C@H]([C@@H]1CCC2=O)CC(=C)C4=CC(=O)C=C[C@]34C",
"C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@@H]2O)[C@@H](CC4=C3C=CC(=C4)O)CCCCCCCCCS(=O)CCCC(C(F)(F)F)(F)F",
"C/C(=C\c1csc(C)n1)[C@@H]1C[C@@H]2O[C@]2(C)CCC[C@H](C)[C@H](O)[C@@H](C)C(=O)C(C)(C)[C@@H](O)CC(=O)N1"
"C1=CC(=CC=C1C#N)C(C2=CC=C(C=C2)C#N)N3C=NC=N3",
"CC(=O)O[C@]1(C(C)=O)CC[C@H]2[C@@H]3C=C(C)C4=CC(=O)CC[C@]4(C)[C@H]3CC[C@@]21C",
"CN(CC1=CN=C2C(=N1)C(=NC(=N2)N)N)C3=CC=C(C=C3)C(=O)N[C@@H](CCC(=O)O)C(=O)O",
"CC/C(=C(\c1ccccc1)c1ccc(OCCN(C)C)cc1)c1ccccc1",
"C1CN1P(=S)(N2CC2)N3CC3",
"CN1CC[C@H](C[C@@H]1C2=NC3=CC=CC=C3N2)NC(=O)NC4=CC=C(C=C4)C#N",
"CC1=C(C(=O)N(C2=NC(=NC=C12)NC3=NC=C(C=C3)N4CCNCC4)C5CCCC5)C(=O)C",
"CCC1=C(N=C(C(=N1)C(=O)N)NC2=CC(=C(C=C2)N3CCC(CC3)N4CCN(CC4)C)OC)NC5CCOCC5",
"C1CC(=O)N(C1C(=O)N(C2=CC(=CN=C2)F)C(C3=CC=CC=C3Cl)C(=O)NC4CC(C4)(F)F)C5=NC=CC(=C5)C#N" ]







# List to store weighted graphs
weighted_graphs = []

# Loop over SMILES strings to generate graphs
for smiles in smiles_list:
    molecule = Chem.MolFromSmiles(smiles)
    graph = nx.Graph()
    for atom in molecule.GetAtoms():
        graph.add_node(atom.GetIdx(), weight=calculate_vertex_weight(atom.GetSymbol()))

    for bond in molecule.GetBonds():
        edge_weight = calculate_edge_weight(bond)
        graph.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), weight=edge_weight)

    weighted_graphs.append(graph)

# Convert each graph to adjacency matrix
adjacency_matrices = [graph_to_adjacency_matrix(graph) for graph in weighted_graphs]


# Function to compute the square of the adjacency matrix
def compute_adjacency_square(adjacency_matrix):
    adjacency_square = np.dot(adjacency_matrix, adjacency_matrix)
    return adjacency_square

# Compute the square of the adjacency matrix for each graph
adjacency_squares = [compute_adjacency_square(adjacency_matrix) for adjacency_matrix in adjacency_matrices]

# Function to extract the diagonal entries of a matrix
def extract_diagonal_entries(matrix):
    diagonal_entries = np.diag(matrix)
    return diagonal_entries


    
 # Function to compute atom degrees based on the adjacency matrix
def compute_atom_degrees(adjacency_matrix):
    # Atom degrees are the diagonal entries of the square adjacency matrix
    atom_degrees = np.diag(adjacency_matrix)
    return atom_degrees

# Compute the atom degrees for each graph
atom_degrees_list = [compute_atom_degrees(adjacency_square) for adjacency_square in adjacency_squares]

# Print the atom degrees for each graph
for i, atom_degrees in enumerate(atom_degrees_list):
    print(f"Graph {i+1} - Atom Degrees:")
    for j, degree in enumerate(atom_degrees):
        print(f"Atom {j+1}: {degree}")
    print()
    

# Function to compute the distance matrix based on edge weights
def compute_distance_matrix(graph):
    num_nodes = len(graph.nodes)
    distance_matrix = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(i, num_nodes):
            if i == j:
                distance_matrix[i][j] = graph.nodes[i]['weight']
            else:
                try:
                    shortest_path_length = nx.shortest_path_length(graph, source=i, target=j, weight='weight')
                    distance_matrix[i][j] = shortest_path_length
                    distance_matrix[j][i] = shortest_path_length  # Since the graph is undirected
                except nx.NetworkXNoPath:
                    pass
    return distance_matrix


# Compute the distance matrix for each graph
distance_matrices = [compute_distance_matrix(graph) for graph in weighted_graphs]


 
    # Function to compute the Wiener index for a given distance matrix
def compute_wiener_index(distance_matrix):
    num_nodes = distance_matrix.shape[0]
    wiener_index = 0
    for i in range(num_nodes):
        for j in range(i):
            wiener_index += distance_matrix[i][j]
    return wiener_index

# Compute the Wiener index for each graph
wiener_indices = [compute_wiener_index(distance_matrix) for distance_matrix in distance_matrices]

# Print the Wiener index for each graph
for i, wiener_index in enumerate(wiener_indices):
    print(f"Graph {i+1} - Wiener Index: {wiener_index}")

# Function to compute atom degrees based on the adjacency matrix
def compute_atom_degrees(adjacency_matrix):
    # Atom degrees are the diagonal entries of the square adjacency matrix
    atom_degrees = np.diag(adjacency_matrix)
    return atom_degrees

# Compute the atom degrees for each graph
atom_degrees_list = [compute_atom_degrees(adjacency_square) for adjacency_square in adjacency_squares]

# Function to compute DD, RDD, and Gutman indices for a given distance matrix
def compute_indices(distance_matrix, atom_degrees_list):
    num_nodes = distance_matrix.shape[0]
    DD = 0
    RDD = 0
    Gutman = 0
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):  # Adjusted to start from i + 1 instead of i
            distance_ij = distance_matrix[i][j]
            if i < len(atom_degrees_list) and j < len(atom_degrees_list):
                DD += (atom_degrees_list[i] + atom_degrees_list[j]) * distance_ij
                RDD += (atom_degrees_list[i] + atom_degrees_list[j]) / distance_ij
                Gutman += atom_degrees_list[i] * atom_degrees_list[j] * distance_ij
    return DD, RDD, Gutman

# Compute the indices for each graph
indices_list = [compute_indices(distance_matrix, atom_degrees_list[i]) for i, distance_matrix in enumerate(distance_matrices)]

# Print the indices for each graph
for i, indices in enumerate(indices_list):
    DD, RDD, Gutman = indices
    print(f"Graph {i+1} - DD: {DD}, RDD: {RDD}, Gutman: {Gutman}")

# Function to compute the eccentricity of each atom
def compute_eccentricity(distance_matrix):
    eccentricities = np.max(distance_matrix, axis=0)
    return eccentricities

# Compute the eccentricity for each graph
eccentricities_list = [compute_eccentricity(distance_matrix) for distance_matrix in distance_matrices]

# Function to compute TEI and ECI
def compute_indices(eccentricities_list, atom_degrees_list):
    TEI_list = []
    ECI_list = []
    for eccentricities, atom_degrees in zip(eccentricities_list, atom_degrees_list):
        TEI = np.sum(eccentricities)
        ECI = np.sum(np.multiply(atom_degrees, eccentricities))
        TEI_list.append(TEI)
        ECI_list.append(ECI)
    return TEI_list, ECI_list

# Compute TEI and ECI
TEI_list, ECI_list = compute_indices(eccentricities_list, atom_degrees_list)

# Print TEI and ECI for each graph
for i, (TEI, ECI) in enumerate(zip(TEI_list, ECI_list)):
    print(f"Graph {i+1} - TEI: {TEI}, ECI: {ECI}")
    
def balaban_index(graph, adjacency_matrix, distance_matrices, i):
    n = len(graph.nodes())
    m = len(graph.edges())
    balaban = 0
    for u, v in graph.edges():
        du =  np.sum(distance_matrices[i][u])
        dv =  np.sum(distance_matrices[i][v])
        balaban += 1 / np.sqrt(du * dv)

    balaban_index = (m / (m - n + 2)) * balaban
    return balaban_index

balaban_indices = [balaban_index(graph, adjacency_matrices[i], distance_matrices, i) for i, graph in enumerate(weighted_graphs)]


# Print Balaban index for each graph
for i, balaban_index in enumerate(balaban_indices):
    print(f"Graph {i+1} - Balaban Index: {balaban_index}")
    
  # Function to calculate Harary index
def harary_index(graph):
    n = len(graph.nodes())
    distance_matrix = nx.floyd_warshall_numpy(graph)
    harary = 0
    for i in range(n):
        for j in range(i+1, n):
            harary += 1 / distance_matrix[i][j]
    return 0.5 * harary

# Calculate Harary index for each graph
harary_indices = [harary_index(graph) for graph in weighted_graphs]

# Print Harary index for each graph
for i, harary_index in enumerate(harary_indices):
    print(f"Graph {i+1} - Harary Index: {harary_index}")


# Create a DataFrame to store the indices
data = {
    "Graph": [],
    "Wiener": [],
    "DD": [],
    "RDD": [],
    "Gutman": [],
    "TEI": [],
    "ECI": [],
    "Balaban": [],
    "Harary": []
}

# Populate the DataFrame with indices data
for i, wiener_index in enumerate(wiener_indices):
    data["Graph"].append(f"Graph {i+1}")
    data["Wiener"].append(wiener_index)
    DD, RDD, Gutman = indices_list[i]
    data["DD"].append(DD)
    data["RDD"].append(RDD)
    data["Gutman"].append(Gutman)
    data["TEI"].append(TEI_list[i])
    data["ECI"].append(ECI_list[i])
    data["Balaban"].append(balaban_indices[i])
    data["Harary"].append(harary_indices[i])
# Create a DataFrame
df = pd.DataFrame(data)  

# Export the DataFrame to Excel
df.to_excel("degreedistanceindices.xlsx", index=False)


<>:168: SyntaxWarning: invalid escape sequence '\['
<>:173: SyntaxWarning: invalid escape sequence '\c'
<>:183: SyntaxWarning: invalid escape sequence '\C'
<>:236: SyntaxWarning: invalid escape sequence '\c'
<>:240: SyntaxWarning: invalid escape sequence '\c'
<>:168: SyntaxWarning: invalid escape sequence '\['
<>:173: SyntaxWarning: invalid escape sequence '\c'
<>:183: SyntaxWarning: invalid escape sequence '\C'
<>:236: SyntaxWarning: invalid escape sequence '\c'
<>:240: SyntaxWarning: invalid escape sequence '\c'
C:\Users\NEVU\AppData\Local\Temp\ipykernel_3576\332523638.py:168: SyntaxWarning: invalid escape sequence '\['
  "C/C=C1\[C@H]2C=C(C)C[C@]1(N)c1ccc(=O)[nH]c1C2",
C:\Users\NEVU\AppData\Local\Temp\ipykernel_3576\332523638.py:173: SyntaxWarning: invalid escape sequence '\c'
  "CC(C)/N=c1\cc2n(-c3ccc(Cl)cc3)c3ccccc3nc-2cc1Nc1ccc(Cl)cc1",
C:\Users\NEVU\AppData\Local\Temp\ipykernel_3576\332523638.py:183: SyntaxWarning: invalid escape sequence '\C'
  "CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(

Graph 1 - Atom Degrees:
Atom 1: 0.7353059439030805
Atom 2: 2.2262241325766965
Atom 3: 0.7353059439030805
Atom 4: 1.7353059439030805
Atom 5: 2.0
Atom 6: 1.7353059439030805
Atom 7: 2.2262241325766965
Atom 8: 2.7353059439030805
Atom 9: 2.0
Atom 10: 2.0
Atom 11: 2.0
Atom 12: 2.0
Atom 13: 2.1403035815518785
Atom 14: 0.6717683508428745
Atom 15: 2.1403035815518785
Atom 16: 2.0
Atom 17: 2.0
Atom 18: 2.114776334924594
Atom 19: 0.551979544866817
Atom 20: 2.0
Atom 21: 2.7353059439030805

Graph 2 - Atom Degrees:
Atom 1: 0.7353059439030805
Atom 2: 2.2262241325766965
Atom 3: 1.7353059439030805
Atom 4: 1.7353059439030805
Atom 5: 2.2262241325766965
Atom 6: 1.7353059439030805
Atom 7: 2.0
Atom 8: 1.7353059439030805
Atom 9: 2.2262241325766965
Atom 10: 2.7353059439030805
Atom 11: 2.0
Atom 12: 2.0
Atom 13: 2.0
Atom 14: 2.0
Atom 15: 2.1403035815518785
Atom 16: 0.6717683508428745
Atom 17: 2.1403035815518785
Atom 18: 2.0
Atom 19: 2.0
Atom 20: 3.0
Atom 21: 2.199123245346685
Atom 22: 0.5349666075410157
Atom 23: